In [1]:
import numpy as np
import pandas as pd

In [12]:
a = pd.read_csv('DATA/a.csv')

In [13]:
a.columns

Index(['Unnamed: 0', 'EntityID', 'Name', 'Status', 'StatusReason',
       'Status Date', 'Duration', 'IncorpDate', 'IncorpState', 'IndustryCode',
       'Street1', 'Street2', 'City', 'State', 'Zip', 'PrinOffEffDate',
       'RA-Name', 'RA-Street1', 'RA-Street2', 'RA-City', 'RA-State', 'RA-Zip',
       'RA-EffDate', 'RA-Status', 'RA-Loc', 'StockInd', 'TotalShares',
       'MergerInd', 'AssessInd', 'Is Series LLC', 'Is Protected Series',
       'Series LLC ID'],
      dtype='object')

In [2]:
df = pd.read_parquet('OUTPUT/evictions_matched.parquet')

In [4]:
df.shape

(515238, 82)

In [7]:
df.head()

,plaintiff_name,defendant_name,defendant_zip,c2dp_case_id,county,fips,case_number,filed_date,case_type,debt_type,...,RA-Status,RA-Loc,StockInd,TotalShares,MergerInd,AssessInd,Is Series LLC,Is Protected Series,Series LLC ID,EntityID_normalized
0,"1 HIL MAR, INC","SCHOTT, HENRY J",23227.0,127_GV22000423-00,New Kent General District Court,127,GV22000423-00,2022-08-18,Unlawful Detainer,None,...,087,,,,1,,,,NaN,ar
1,"1 HIL MAR, INC","SCHOTT, HENRY J",23227.0,127_GV22000423-00,New Kent General District Court,127,GV22000423-00,2022-08-18,Unlawful Detainer,None,...,153,,,,1,,,,NaN,ar
2,"1 HIL MAR, INC","SCHOTT, HENRY J",23227.0,127_GV22000423-00,New Kent General District Court,127,GV22000423-00,2022-08-18,Unlawful Detainer,None,...,107,,,,1,,,,NaN,ar
3,"1 HIL MAR, INC","SCHOTT, HENRY J",23227.0,127_GV22000423-00,New Kent General District Court,127,GV22000423-00,2022-08-18,Unlawful Detainer,None,...,600,,,,1,,,,NaN,ar
4,1 SQ APTS LP,AYALA ERICA,23702.0,740_GV18012249-00,Portsmouth General District Court,740,GV18012249-00,2018-11-21,Unlawful Detainer,None,...,,,,,1,,,,NaN,ap


In [3]:
df.columns

Index(['plaintiff_name', 'defendant_name', 'defendant_zip', 'c2dp_case_id',
       'county', 'fips', 'case_number', 'filed_date', 'case_type', 'debt_type',
       'defendant_attorney', 'defendant_dba_ta', 'defendant_address',
       'defendant_count', 'all_defendant_names', 'all_defendant_addresses',
       'defendant_order_validation', 'hearing_count', 'latest_hearing_date',
       'latest_hearing_result', 'judgment', 'homestead_exemption_waived',
       'is_judgment_satisfied', 'date_satisfaction_filed',
       'further_case_information', 'costs', 'attorney_fees',
       'principal_amount', 'interest_award', 'possession',
       'writ_of_eviction_issued_date', 'other_amount', 'other_awarded',
       'clean_party_name', 'plaintiff_attorney', 'plaintiff_dba_ta',
       'plaintiff_address', 'plaintiff_count', 'all_plaintiff_names',
       'all_plaintiff_addresses', 'plaintiff_order_validation', 'filed_year',
       'filed_quarter', 'plaintiff_zip', 'serial_filing',
       'latest_filing

In [15]:
df['Zip'].head()

0    2014-06-19
1    2019-10-23
2    2021-05-19
3    2022-12-12
4    1994-01-31
Name: Zip, dtype: object

Address-based clustering

In [38]:
plaintiff_col = "plaintiff_name"
matched_col = "EntityID" 

# Sample 20 random rows where a match was found
sample = df[[plaintiff_col, matched_col]].dropna().sample(n=20)


In [8]:
for _, row in sample.iterrows():
    print(f"Filed Plaintiff: {row[plaintiff_col]}\nMatched LLC:     {row[matched_col]}\n{'-'*60}")

Filed Plaintiff: KPM LLC
Matched LLC:     KPM LLC   
------------------------------------------------------------
Filed Plaintiff: RBG BEACON 303 ASSOCIATES LLC
Matched LLC:     RBG BEACON 303 ASSOCIATES, LLC
------------------------------------------------------------
Filed Plaintiff: POLIZOS PROPERTIES LLC
Matched LLC:     POLIZOS PROPERTIES LLC
------------------------------------------------------------
Filed Plaintiff: CRS BROOKMONT HOUSING CORP
Matched LLC:     J&P HOUSING LLC
------------------------------------------------------------
Filed Plaintiff: WAVERTON CS DEL OWNER LLC
Matched LLC:     Waverton CS Del Owner, LLC
------------------------------------------------------------
Filed Plaintiff: COMMONS AT PLANT ZERO LLC, THE
Matched LLC:     The Commons at Plant Zero, LLC
------------------------------------------------------------
Filed Plaintiff: WHISPERING OAKS APTS LP
Matched LLC:     Salem Colony, LLC - 841 Whispering Woods Court, Protected Series
-----------------------

In [34]:
grouped_matches = (
    df
    .groupby(matched_col)[plaintiff_col]
    .nunique()
    .sort_values(ascending=False)
)

# Show the top 20 matched names by diversity of inputs
print(grouped_matches.head(20))

EntityID
MGM, LLC                  172
RPMC, LLC                 128
RLP, LLC                  128
RCO LLC                   128
MACO LLC                  128
Property Property, LLC    116
A.P., LLC                  79
P&R LLC                    78
ERS LLC                    73
E.R.S. LLC                 73
ATES, LLC                  72
c² LLC                     68
L & C LLC                  68
C LLC                      68
L & C, L.L.C.              68
Investments L.L.C.         61
LandL, LLC                 59
P.P.J.C., L.C.             52
ITED LLC                   47
DIN, LLC                   45
Name: plaintiff_name, dtype: int64


Starting to play around with more matching

In [39]:
# Work with unique entities only
df = (
    df
    .sort_values('filed_date', ascending=False)
    .groupby('plaintiff_name_normalized', as_index=False)
    .first()
)

In [20]:
# Starting with >90 confidence
df = df.loc[df['match_confidence'] > 89]

In [21]:
df.shape

(15238, 81)

In [40]:
## More text normalization
def clean_name(df, source_col, target_col):
    ndf = df.copy()
    ndf[target_col] = (
        ndf[source_col]
        .fillna('')
        .str.lower()
        .str.replace(r'[^\w\s\.-]', '', regex=True)
        .replace(r'\s+', ' ', regex=True)
        .str.strip()
    )
    return ndf


In [41]:
# RA Names
df = clean_name(df, 'RA-Name', 'clean_ra_name')

# Remove common terms
ra_terms = ['registered agent', 'agent for service', 'agent']
ra_pattern = '|'.join(ra_terms)
df['clean_ra_name'] = (
    df['clean_ra_name']
    .str.replace(ra_pattern, '', regex=True)
    .str.strip()
)

In [42]:
def clean_address_cols(df, source_cols):
    ndf = df.copy()
    for source_cols, target_col in source_cols.items():
        ndf[target_col] = ndf[list(source_cols)].fillna('').agg(' '.join, axis=1)
        ndf[target_col] = clean_address_series(ndf[target_col])
    return ndf

def clean_address_series(series):
    clean = series.str.lower()
    replacements = {
        ' street ': ' st ', ' avenue ': ' ave ', ' boulevard ': ' blvd ', ' road ': ' rd ',
        ' drive ': ' dr ', ' lane ': ' ln ', ' suite ': ' ste ', ' apartment ': ' apt ',
        ' north ': ' n ', ' south ': ' s ', ' east ': ' e ', ' west ': ' w '
    }
    for original, replacement in replacements.items():
        clean = clean.str.replace(original, replacement)
    clean = clean.str.replace(r'[^\w\s]', '', regex=True)
    clean = clean.str.replace(r'\s+', ' ', regex=True).str.strip()
    return clean


In [43]:
## Addresses
address_mappings = {
        # Principal address components
        ('Street1', 'Street2', 'City', 'State', 'Zip'): 'clean_principal_address',
        # RA address components
        ('RA-Street1', 'RA-Street2', 'RA-City', 'RA-State', 'RA-Zip'): 'clean_ra_address'
    }

df = clean_address_cols(df, address_mappings)

In [10]:
def create_initial_clusters(df):
    """Create initial clusters based on exact matching"""
    # Initialize empty series for cluster IDs with default value -1
    cluster_ids = pd.Series(index=df.index, data=-1)
    current_cluster_id = 0
    cluster_reasons = pd.Series(index=df.index, data="")
    
    # Group by registered agent - vectorized approach
    if 'clean_ra_name' in df.columns:
        # Filter out empty RAs and group
        ra_groups = df[df['clean_ra_name'] != ''].groupby('clean_ra_name')
        
        # Find groups with more than one entity
        for ra_name, group in ra_groups:
            if len(group) > 1:
                # Assign cluster ID to all entities in this group
                cluster_ids.loc[group.index] = current_cluster_id
                cluster_reasons.loc[group.index] = "RA"
                current_cluster_id += 1
    
    # Group by address for remaining unassigned entities
    if 'clean_principal_address' in df.columns:
        # Filter to only process unassigned entities (-1) with non-empty addresses
        unassigned_mask = (cluster_ids == -1) & (df['clean_principal_address'] != '')
        
        if unassigned_mask.any():
            address_groups = df[unassigned_mask].groupby('clean_principal_address')
            
            # Find groups with more than one entity
            for address, group in address_groups:
                if len(group) > 1:
                    # Assign cluster ID to all entities in this group
                    cluster_ids.loc[group.index] = current_cluster_id
                    cluster_reasons.loc[group.index] = "Address"
                    current_cluster_id += 1
    
    # Add the cluster IDs and reasons to the DataFrame
    result_df = df.copy()
    result_df['cluster_id'] = cluster_ids
    result_df['cluster_reason'] = cluster_reasons
    
    # Calculate some basic statistics
    cluster_stats = {
        'total_entities': len(df),
        'clustered_entities': (cluster_ids != -1).sum(),
        'unclustered_entities': (cluster_ids == -1).sum(),
        'unique_clusters': cluster_ids[cluster_ids != -1].nunique()
    }
    
    return result_df, cluster_stats

In [11]:
df_clustered, stats = create_initial_clusters(df)

In [12]:
print(f"Total entities: {stats['total_entities']}")
print(f"Entities in clusters: {stats['clustered_entities']} ({stats['clustered_entities']/stats['total_entities']:.1%})")
print(f"Entities not in clusters: {stats['unclustered_entities']} ({stats['unclustered_entities']/stats['total_entities']:.1%})")
print(f"Unique clusters: {stats['unique_clusters']}")


Total entities: 15238
Entities in clusters: 9778 (64.2%)
Entities not in clusters: 5460 (35.8%)
Unique clusters: 1656


In [13]:
def analyze_top_clusters(df, n=10):
    """Analyze the top N clusters by size"""
    # Find the largest clusters
    top_cluster_ids = df[df['cluster_id'] != -1]['cluster_id'].value_counts().nlargest(n).index
    
    cluster_details = []
    
    for cid in top_cluster_ids:
        cluster_df = df[df['cluster_id'] == cid]
        size = len(cluster_df)
        
        # Get the reason for clustering
        reason = cluster_df['cluster_reason'].iloc[0]
        
        # Get the common elements
        if reason == "RA":
            common_element = cluster_df['clean_ra_name'].iloc[0]
        elif reason == "Address":
            common_element = cluster_df['clean_principal_address'].iloc[0]
        else:
            common_element = "Unknown"
        
        # Count unique attributes
        unique_addresses = cluster_df['clean_principal_address'].nunique()
        unique_ras = cluster_df['clean_ra_name'].nunique()
        
        # Sample of entity names
        sample_entities = cluster_df['Name'].sample(min(5, size)).tolist()
        
        # Count of eviction filings if available
        eviction_count = 0
        if 'serial_filing' in cluster_df.columns:
            eviction_count = cluster_df['serial_filing'].sum()
        
        cluster_details.append({
            'cluster_id': cid,
            'size': size,
            'reason': reason,
            'common_element': common_element,
            'unique_addresses': unique_addresses,
            'unique_ras': unique_ras,
            'sample_entities': sample_entities,
            'eviction_count': eviction_count
        })
    
    return pd.DataFrame(cluster_details)


In [14]:
top_clusters = analyze_top_clusters(df_clustered, n=10)
print(top_clusters[['cluster_id', 'size', 'reason', 'unique_addresses', 'unique_ras']])


   cluster_id  size   reason  unique_addresses  unique_ras
0          22   566       RA               436           1
1        1051   510       RA               367           1
2         594   169       RA                 2           1
3        1436   151       RA               126           1
4        1014   139       RA                11           1
5        1358   115       RA                 1           1
6        1058   109       RA                59           1
7        1341    94       RA                64           1
8         525    79       RA                 8           1
9        1611    78  Address                 1           1


In [16]:
cluster_22 = df_clustered.loc[df_clustered['cluster_id']==22]
cluster_22['RA-Name'].head()

plaintiff_name_normalized
1 2020 borrower hpa llc ml                    100 Shockoe Slip Fl 2
1 acquisitions hpa                            100 Shockoe Slip Fl 2
1 armstrong church hill llc north phase ta    100 Shockoe Slip Fl 2
1 block llc pointe summit                     100 Shockoe Slip Fl 2
1 cox landing llc tic                         100 Shockoe Slip Fl 2
Name: RA-Name, dtype: object

In [44]:
def identify_likely_related_by_name(df):
    """Identify likely related entities based on name similarity patterns"""
    from difflib import SequenceMatcher
    import re
    
    # Create a dictionary to store likely related pairs
    likely_related = {}
    
    # Get all entity names
    entities = df[['EntityID_normalized', 'plaintiff_name_normalized']].values
    
    # Patterns that strongly suggest relation
    sequential_patterns = [
        r"(.*)\s+[I|V|X]$",      # Entities with Roman numerals
        r"(.*)\s+#\d+$",         # Entities with sequential numbering
        r"(.*)\s+\d+$",          # Properties with numbers
        r"(.*)\s+(?:LLC|LP|Inc)$"  # Same name with different entity types
    ]
    
    # Group by naming patterns
    pattern_groups = {}
    for pattern in sequential_patterns:
        for entity_id, name in entities:
            match = re.match(pattern, name)
            if match:
                base_name = match.group(1).strip()
                if base_name not in pattern_groups:
                    pattern_groups[base_name] = []
                pattern_groups[base_name].append((entity_id, name))
    
    # Add all pairs that match patterns
    for base_name, group in pattern_groups.items():
        if len(group) > 1:
            likely_related[base_name] = [entity_id for entity_id, _ in group]
    
    # For entities without obvious patterns, use string similarity
    # Group entities by first word to reduce comparison space
    first_word_groups = {}
    for entity_id, name in entities:
        if not any(entity_id in group for group in likely_related.values()):
            first_word = name.split()[0].lower() if name and ' ' in name else name.lower()
            if first_word not in first_word_groups:
                first_word_groups[first_word] = []
            first_word_groups[first_word].append((entity_id, name))
    
    # Check similarity within these groups
    for word, group in first_word_groups.items():
        if len(group) > 1:
            for i in range(len(group)):
                for j in range(i+1, len(group)):
                    id1, name1 = group[i]
                    id2, name2 = group[j]
                    similarity = SequenceMatcher(None, name1.lower(), name2.lower()).ratio()
                    if similarity > 0.8:  # High similarity threshold
                        key = f"similar_{word}_{i}_{j}"
                        likely_related[key] = [id1, id2]
    
    return likely_related

In [46]:
df.shape

(15564, 85)

In [47]:
a = df.sample(frac=0.1)

In [48]:
x = identify_likely_related_by_name(df)

AttributeError: 'dict' object has no attribute 'head'